# ETL for the scoring data (questions and corresponding links to sources from UC IPM)

In [ ]:
from urllib.parse import urlparse

def _uri_validator(x):
    try:
        result = urlparse(x)
        return all([result.scheme, result.netloc])
    except:
        return False

In [ ]:
import pandas as pd

UCIPM_DATA  = './data/raw/UCIPM_Chatbot_Core_Questions 2020_08_19.xlsx - Questions with correct link.csv'
AE_DATA     = './data/raw/AE_test_QA_chatbot.xlsx - Questions with correct link.csv'
FINAL_DATA  = './data/transformed/result.pkl'

df = pd.read_csv(UCIPM_DATA)

df.sample(10)

In [ ]:
print(f'Size of DF (before): {df.shape}')
df = df[df['Crawled'] == 'Y']
print(f'Size of DF (after ): {df.shape}')


df['Question'   ] = df['Question'   ].apply(lambda x: x.split('\n')[0]                              )
df['URL'        ] = df['URL'        ].apply(lambda x: x.split('\n')                                 )
df['URL'        ] = df['URL'        ].apply(lambda x: [x1.split('?')[0] for x1 in x]                )

for urls in df['URL']:
    for u in urls:
        assert urlparse(u)

df['ExpectedAnswer' ] = df['Expected answer reference(s)'   ]
df['ExpectedAnswer' ] = df['ExpectedAnswer'                 ].apply(lambda x: x.split('\n'))


df['Source'         ] = 'UCIPM'

ucipm = df[['Question', 'ExpectedAnswer', 'URL', 'Source']]

ucipm.sample(10)

In [ ]:
df = pd.read_csv(AE_DATA)

df.sample(10)

In [ ]:
print(f'Size of DF (before): {df.shape}')
df = df[df['URL'].notna()       ]
df = df[df['URL'] != 'no_answer']
print(f'Size of DF (after ): {df.shape}')

df['URL'] = df['URL'].apply(lambda x: x.split('\n')                 )
df['URL'] = df['URL'].apply(lambda x: [x1.split('?')[0] for x1 in x])

for urls in df['URL']:
    for u in urls:
        assert urlparse(u)

df['Source'] = 'AE'

ae = df[['Question', 'URL', 'Source']]

ae.sample(10)

In [ ]:
df = pd.concat([ucipm, ae])

df.to_pickle(FINAL_DATA)
print(f'Shape of final DF: {df.shape}')
df.sample(10)

In [ ]:
df = pd.read_pickle(FINAL_DATA)
df